# Using this Notebook to scrape Soundcloud to get a list of users that someone is following and who is following that use

Soundcloud has 176 million users. Let's try to capture that

In [21]:
import requests
import pandas as pd
import pprint

### This is for getting the list of people that a user is following

#### These are all the functions that need to be ran before the notebook works:
1. Getting user information (this is used for following and follower list)

In [29]:
# user_id = '58899599'

# following_url = f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'


# following_parsed = requests.get(following_url).json()
# print(following_parsed.keys())
# print(following_parsed.get('next_href'))


# following = get_user_info(following_parsed)

# follower_url = f'https://api-v2.soundcloud.com/users/{user_id}/followers?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535640551&app_locale=en'
# follower_parsed = requests.get(follower_url).json()
# followers = get_user_info(follower_parsed)


In [31]:
def get_user_info(user_id, list_subject):
    """
    this function gets all the information for the json file that is parsed from requests.
    list_subject tells me whether they want following list or follower list
    I need to also account for the fact that next_href can be NOT None so I need to rerun the fcn until that is true
    """
    
    
    url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'
    parsed = requests.get(url).json()
    following_list = []
    base_url = ''
    
    while parsed.get('next_href') != None: 
        following_list.extend(parsed.get('collection'))
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset={offset}&linked_partitioning=1&app_version=1535972763&app_locale=en'
        parsed = requests.get(url).json()

    following_list.extend(parsed.get('collection'))
    print(len(following_list))
    return following_list #this is the collection list which includes all the users in the list_subject

Here's the plan:
- everytime I get someone's follower's, i'm going to put their user_id into a list.
- before i get someone's followers, i need to find if that user_id is already in the list
- - and if not, then i will scrape. if so, then i will not scrape

In [ ]:
# here is a template of how to write things to a csv

# import csv
# RESULT = ['apple','cherry','orange','pineapple','strawberry']
# with open("output.csv",'wb') as resultFile:
#     wr = csv.writer(resultFile, dialect='excel')
#     wr.writerow(RESULT)

In [90]:
# i need to keep track of all the people that i've already scraped

following = get_user_info('38611053', 'followers')
got_followers = []
got_followers.append('38611053')
#print(type(got_followers))
got_followers_df = pd.DataFrame(got_followers).to_pickle('got_followers.pkl') # this is the file where I am storing scraped
#got_followers_df

212


In [15]:
mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count', 'permalink',\
             'reposts_count', 'track_count', 'uri']

following_df = pd.DataFrame(following)[mask]

In [16]:
mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count', 'permalink',\
             'reposts_count', 'track_count', 'uri']


following_df
# things that I want: followers_count, followings_count, id, likes_count, permalink, reposts_count, track_count, uri

,city,country_code,followers_count,followings_count,id,likes_count,permalink,reposts_count,track_count,uri
0,None,None,7,6,477797010,7,notatyourparty,None,0,https://api.soundcloud.com/users/477797010
1,"garden grove, ca",None,9,34,496307469,1,prettylit,None,1,https://api.soundcloud.com/users/496307469
2,None,None,6,10,176660107,10,brandon-bun-699663406,None,0,https://api.soundcloud.com/users/176660107
3,None,None,1,3,440219016,1,oliver-trombetti,None,0,https://api.soundcloud.com/users/440219016
4,None,None,19,8,276222017,3,user-713335820,None,18,https://api.soundcloud.com/users/276222017
5,Vancouver,CA,1439,531,202142770,203,ngynmusic,None,18,https://api.soundcloud.com/users/202142770
6,None,None,66,66,26437110,49,claudio-labra,None,0,https://api.soundcloud.com/users/26437110
7,None,None,9,328,416767671,623,whiskeythanyourtea,None,0,https://api.soundcloud.com/users/416767671
8,None,None,0,3,429935160,2,sdred-abramowitz,None,0,https://api.soundcloud.com/users/429935160
9,None,None,7,31,116322161,1,jennifer-phvm,None,0,https://api.soundcloud.com/users/116322161


In [70]:
# list(following_df.id)

In [71]:
list_of_users_seen = []
list_of_users_seen.extend(list(following_df.id))
list_of_users_seen = set(list_of_users_seen) #doing set makes sure that i dont have duplicates
len(list_of_users_seen)

212

In [72]:
# test_list = []
# test_list.extend(['a', 'b', 'c'])
# test_list.extend(['a', 'b', 'c'])

In [111]:
import numpy as np
got_followers= np.array([])

### This is for getting the list of followers of a user:

In [ ]:
# test
import pickle
followers = get_user_info('38611053','followers')
follower_df = pd.DataFrame(followers)[mask]
got_followers = np.append(got_followers,'38611053')
print(got_followers)
got_followers_df = pd.DataFrame(got_followers).to_pickle('got_followers.pkl')


follower_df

In [1]:
from pymongo import MongoClient
# !sudo apt-get install -y mongodb-org

In [2]:
!sudo service mongod start


# move things into the env file

In [30]:
# !mongo -u mongo_user -p password 127.0.0.1/cool

In [3]:
# i need to put this into a .env file so that i dont get blocked like billy did :(((
# user_followers is the database that i created to put my information into it

connection_string = 'collection_string'

client = MongoClient(connection_string)

In [4]:
#this is me creating a newdatabase

database = client.user_followers

followers = database.followers
following = database.following

The format of followers is that I will start with the source node, destination node, and the relationship is 'following'

In [29]:
# list(following.find())

[]

In [11]:
# following.insert_one({'name': 'jenn', 'who': 'everyone'})
#followers.insert_one({'name': 'brett', 'who': 'everyone'})

In [12]:
db = client.user_followers
followers = db.user_followers

In [26]:
db.list_collection_names()

['following', 'user_followers', 'followers']

In [10]:
followers.delete_one({'name': 'brett',
                 'who': 'everyone'})

In [28]:
delete_this = list(user_followers.find())
delete_this

NameError: name 'user_followers' is not defined

In [24]:
for record in delete_this:
    following.delete_one(record)

In [16]:
# !sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 0C49F3730359A14518585931BC711F9BA15703C6
# !echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list

# !sudo apt-get update
# !sudo apt-get install -y mongodb-org